# Getting Started with CAMEL Workforce and Nebius AI Studio

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/19lIjJEnqK5H1H13YRrUZ-LiTe2DmS_4w#scrollTo=colab_link)


<div class="align-center">
  <a href="https://www.camel-ai.org/"><img src="https://i.postimg.cc/KzQ5rfBC/button.png"width="150"></a>
  <a href="https://discord.camel-ai.org"><img src="https://i.postimg.cc/L4wPdG9N/join-2.png"  width="150"></a></a>
  
⭐ <i>Star us on [*Github*](https://github.com/camel-ai/camel), join our [*Discord*](https://discord.camel-ai.org) or follow our [*X*](https://x.com/camelaiorg)
</div>

## 🚀 Overview  

This notebook shows how to integrate **Nebius AI Studio** models with **CAMEL agents**.  
With a unified **OpenAI-compatible API**, Nebius AI Studio unlocks access to state-of-the-art LLMs including **GPT-OS**, **DeepSeek**, **Llama**, and more.  

---

## 🔎 What is Nebius AI Studio?  

[Nebius AI Studio](https://nebius.com/docs/ai-studio/api) is a **cloud-native platform** providing enterprise-grade access to advanced LLMs.  
It combines **scalability, reliability, and diversity of models** into a single developer-friendly interface.  

---

## 📚 In this guide, you'll explore:  

1. **Simple Testing** → Quick model integration & validation  
2. **Speed Comparison** → Benchmarking performance across Nebius models  
3. **[Workforce](https://docs.camel-ai.org/key_modules/workforce) Integration** → Multi-agent collaboration with CAMEL  


## 📦 Installation

Ensure you have CAMEL-AI installed in your Python environment:

In [ ]:
!pip install "camel-ai[all]==0.2.75"
%pip install nest_asyncio

import nest_asyncio
nest_asyncio.apply()

## 🔑 Setting Up Nebius API Keys

You'll need to set up your API keys for Nebius AI Studio. First, obtain your API key from [Nebius AI Studio](https://studio.nebius.com/).

In [ ]:
import os
from getpass import getpass

# Prompt for the API key securely
nebius_api_key = getpass('Enter your Nebius API key: ')
os.environ["NEBIUS_API_KEY"] = nebius_api_key

print("✅ Nebius API key configured successfully!")

Enter your Nebius API key: ··········
✅ Nebius API key configured successfully!


## 📚 Import Required Libraries

In [ ]:
import time
import asyncio
from typing import List, Dict, Any

from camel.agents import ChatAgent
from camel.configs import NebiusConfig
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.societies.workforce import Workforce
from camel.tasks.task import Task
from camel.messages import BaseMessage

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## 🧪 Section 1: Simple Testing

Let's start with basic integration testing to ensure everything works correctly.

In [ ]:
# Create a Nebius model with custom configuration
# Note: Use the full model name format "openai/gpt-oss-20b"
model = ModelFactory.create(
    model_platform=ModelPlatformType.NEBIUS,
    model_type="openai/gpt-oss-20b",  # Using the correct format
    model_config_dict=NebiusConfig(
        temperature=0.7,
        max_tokens=1000,
        top_p=0.9
    ).as_dict(),
)

In [ ]:
# Define system message
sys_msg = "You are an expert AI researcher specializing in multi-agent systems and autonomous agents."

# Create agent
camel_agent = ChatAgent(
    system_message=sys_msg,
    model=model,
    message_window_size=10
)

# Interact with the agent
user_msg = "Explain the key benefits of using multi-agent systems in AI applications."

response = camel_agent.step(user_msg)
print(response.msgs[0].content)

**Multi‑Agent Systems (MAS)** are collections of autonomous, interacting software or robotic entities that perceive their environment, make decisions, and act to achieve individual or collective goals. Over the last two decades, MAS have become a cornerstone of many AI applications. Below are the core benefits that make them attractive, along with concrete examples of where each advantage shines.

| # | Benefit | What It Means | Typical Use‑Case | Why It Matters |
|---|---------|---------------|------------------|----------------|
| 1 | **Scalability & Parallelism** | Agents can operate concurrently, distributing workload across many cores or nodes. | • Distributed search (e.g., web crawling, protein folding). <br>• Multi‑robot exploration (search‑and‑rescue). | Large‑scale problems that would stall a single process can be tackled by a swarm of lightweight agents, each doing a piece of the job. |
| 2 | **Modularity & Composability** | Each agent encapsulates a specific function or know

## ⚡ Section 2: Model Performance Comparison

Let's compare different Nebius models on the same task:

In [ ]:
# === Nebius model speed + output benchmark ===
import time
from textwrap import shorten
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.configs import NebiusConfig
from camel.types import ModelPlatformType

def make_nebius_model(model_type: str):
    return ModelFactory.create(
        model_platform=ModelPlatformType.NEBIUS,
        model_type=model_type,
        model_config_dict=NebiusConfig(
            temperature=0.3,
            max_tokens=280,
            top_p=0.9,
        ).as_dict(),
    )

def benchmark_model(model_type: str, model_name: str, prompt: str, runs: int = 3):
    """Benchmark latency and return outputs for each run."""
    try:
        model = make_nebius_model(model_type)
        agent = ChatAgent(
            system_message="You are a helpful assistant. Provide concise responses.",
            model=model,
        )
        times, outputs = [], []

        for i in range(runs):
            t0 = time.time()
            resp = agent.step(prompt)
            dt = time.time() - t0
            text = resp.msgs[0].content
            times.append(dt)
            outputs.append(text)

        return {
            "model_name": model_name,
            "avg_time": sum(times) / len(times),
            "min_time": min(times),
            "max_time": max(times),
            "times": times,
            "outputs": outputs,
            "success": True,
            "error": None,
        }
    except Exception as e:
        return {
            "model_name": model_name,
            "avg_time": float("inf"),
            "min_time": None,
            "max_time": None,
            "times": [],
            "outputs": [],
            "success": False,
            "error": str(e)[:200],
        }

In [ ]:
# ---- Configure models ----
models_to_benchmark = [
    ("openai/gpt-oss-20b", "GPT-OS 20B"),
    ("deepseek-ai/DeepSeek-V3", "DeepSeek V3"),
    ("meta-llama/Meta-Llama-3.1-70B-Instruct", "Llama 3.1 70B"),
]

prompt = "Explain the concept of machine learning in 2 sentences."
runs = 3

print("🚀 Starting speed + output comparison...\n" + "="*60)

results = []
for mtype, mname in models_to_benchmark:
    print(f"\n🔎 Testing {mname}:")
    r = benchmark_model(mtype, mname, prompt, runs=runs)
    results.append(r)
    if r["success"]:
        for i, (t, out) in enumerate(zip(r["times"], r["outputs"]), 1):
            preview = shorten(out.replace("\n", " "), width=100, placeholder="…")
            print(f"  • Run {i}: {t:.2f}s | {preview}")
    else:
        print(f"  ⚠️ Error: {r['error']}")

# ---- Summary sorted by average time ----
ok = [r for r in results if r["success"]]
ok.sort(key=lambda x: x["avg_time"])

print("\n" + "="*60)
print("🏁 SPEED COMPARISON SUMMARY (avg / min / max)")
for i, r in enumerate(ok, 1):
    print(f"{i}. {r['model_name']}: {r['avg_time']:.2f}s avg "
          f"(min {r['min_time']:.2f}s, max {r['max_time']:.2f}s)")

# ---- Show full outputs of the fastest model ----
if ok:
    fastest = ok[0]
    print("\n" + "="*60)
    print(f"📝 FULL OUTPUTS — Fastest: {fastest['model_name']}")
    for i, out in enumerate(fastest["outputs"], 1):
        print(f"\n--- Output Run {i} ---\n{out}")

🚀 Starting speed + output comparison...

🔎 Testing GPT-OS 20B:
  • Run 1: 1.00s | Machine learning is a branch of artificial intelligence that enables computers to learn patterns…
  • Run 2: 1.01s | Machine learning is a field of AI where algorithms learn to recognize patterns and make predictions…
  • Run 3: 0.92s | Machine learning is a subset of artificial intelligence that trains algorithms to identify patterns…

🔎 Testing DeepSeek V3:
  • Run 1: 3.46s | Machine learning is a subset of artificial intelligence that enables systems to learn and improve…
  • Run 2: 2.00s | Machine learning is a field of artificial intelligence where algorithms learn patterns from data to…
  • Run 3: 1.65s | Machine learning is a branch of AI that allows computers to learn from data and improve their…

🔎 Testing Llama 3.1 70B:
  • Run 1: 1.97s | Machine learning is a type of artificial intelligence that enables computers to learn from data and…
  • Run 2: 1.69s | Machine learning is a subset of artific

## 👥👥 Section 3: Workforce Integration  

We’ll now add **Workforce**, CAMEL-AI’s module for multi-agent collaboration, to use Nebius models together.  
This lets us combine multiple specialized agents (e.g., researcher, reviewer, summarizer) and coordinate them on a shared task.  

🔗 Learn more in the [Workforce docs](https://docs.camel-ai.org/key_modules/workforce).  


In [ ]:
# --- 3-agent workflow using 3 different Nebius models ---

def build_nebius_model(model_type):
    if not os.environ.get("NEBIUS_API_KEY"):
        raise RuntimeError("Set NEBIUS_API_KEY before running.")
    return ModelFactory.create(
        model_platform=ModelPlatformType.NEBIUS,
        model_type=model_type,
        model_config_dict=NebiusConfig(temperature=0.2).as_dict(),
    )

# Create 3 agents with different models
researcher = ChatAgent(
    system_message=BaseMessage.make_assistant_message(
        role_name="Researcher",
        content="You are a knowledgeable researcher. Write a comprehensive paragraph (4-6 sentences) with factual information and key details.",
    ),
    model=build_nebius_model("openai/gpt-oss-20b"),
)

summarizer = ChatAgent(
    system_message=BaseMessage.make_assistant_message(
        role_name="Summarizer",
        content="You are an expert summarizer. Create a concise 2-3 sentence summary that captures the main points.",
    ),
    model=build_nebius_model("deepseek-ai/DeepSeek-V3"),
)

tweet_writer = ChatAgent(
    system_message=BaseMessage.make_assistant_message(
        role_name="Tweet Writer",
        content="You are a social media expert. Write an engaging tweet (max 280 chars) that's informative and uses relevant hashtags.",
    ),
    model=build_nebius_model("meta-llama/Meta-Llama-3.1-70B-Instruct"),
)

# Sequential execution
print("🔍 Step 1: Researching with GPT-OS 20B...")
research_response = researcher.step("Write a detailed paragraph about quantum computing, its current state, and potential applications.")
research_text = research_response.msgs[0].content
print(f"Research: {research_text}\n")

print("📝 Step 2: Summarizing with DeepSeek V3...")
summary_response = summarizer.step(f"Summarize this research: {research_text}")
summary_text = summary_response.msgs[0].content
print(f"Summary: {summary_text}\n")

print("🐦 Step 3: Creating tweet with Llama 3.1 70B...")
tweet_response = tweet_writer.step(f"Based on this summary: {summary_text}, write an engaging tweet about quantum computing.")
tweet_text = tweet_response.msgs[0].content
print(f"Tweet: {tweet_text}\n")

print("✅ All steps completed with 3 different models!")

🔍 Step 1: Researching with GPT-OS 20B...
Research: Quantum computing harnesses the principles of superposition and entanglement to perform calculations that would be infeasible for classical machines, using quantum bits (qubits) that can represent both 0 and 1 simultaneously. As of 2025, leading research labs and companies such as IBM, Google, and Rigetti have built processors with 50–100 qubits, and IBM’s 433‑qubit Eagle system demonstrates error‑corrected logical qubits, marking a significant step toward fault‑tolerant operation. Despite these advances, practical quantum advantage remains limited to specialized tasks—Google’s Sycamore achieved “quantum supremacy” on a random circuit sampling problem, while D-Wave’s quantum annealers excel at certain combinatorial optimization problems. Potential applications span cryptography (e.g., Shor’s algorithm for factoring large integers), drug discovery and materials science (through quantum simulation of molecular systems), and machine learn

## 🌟 Highlights

You’ve seen how to leverage **Nebius AI Studio** within **CAMEL** for real-world tasks:

- **Simple Testing** – Validate model integration
- **Speed Comparison** – Benchmarks across Nebius models
- **Workforce Integration** – Coordinated multi-agent workflows

---

## ​ What’s Next?

-  Dive into the [CAMEL-AI Documentation](https://docs.camel-ai.org/get_started/introduction) to master agents, tools, societies, and orchestration patterns like Workforce.
- Try more models in the [Nebius AI Studio Playground](https://nebius.com/ai-studio) and prototype real-time agent behaviors.


---

That's everything: Got questions about 🐫 CAMEL-AI? Join us on [Discord](https://discord.camel-ai.org)! Whether you want to share feedback, explore the latest in multi-agent systems, get support, or connect with others on exciting projects, we’d love to have you in the community! 🤝

Check out some of our other work:

1. 🐫 Creating Your First CAMEL Agent [free Colab](https://docs.camel-ai.org/cookbooks/create_your_first_agent.html)

2.  Graph RAG Cookbook [free Colab](https://colab.research.google.com/drive/1uZKQSuu0qW6ukkuSv9TukLB9bVaS1H0U?usp=sharing)

3. 🧑‍⚖️ Create A Hackathon Judge Committee with Workforce [free Colab](https://colab.research.google.com/drive/18ajYUMfwDx3WyrjHow3EvUMpKQDcrLtr?usp=sharing)

4. 🔥 3 ways to ingest data from websites with Firecrawl & CAMEL [free Colab](https://colab.research.google.com/drive/1lOmM3VmgR1hLwDKdeLGFve_75RFW0R9I?usp=sharing)

5. 🦥 Agentic SFT Data Generation with CAMEL and Mistral Models, Fine-Tuned with Unsloth [free Colab](https://colab.research.google.com/drive/1lYgArBw7ARVPSpdwgKLYnp_NEXiNDOd-?usp=sharingg)

Thanks from everyone at 🐫 CAMEL-AI


<div class="align-center">
  <a href="https://www.camel-ai.org/"><img src="https://i.postimg.cc/KzQ5rfBC/button.png"width="150"></a>
  <a href="https://discord.camel-ai.org"><img src="https://i.postimg.cc/L4wPdG9N/join-2.png"  width="150"></a></a>
  
⭐ <i>Star us on [*Github*](https://github.com/camel-ai/camel), join our [*Discord*](https://discord.camel-ai.org) or follow our [*X*](https://x.com/camelaiorg)
</div>

